<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/GenAI/BasicLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Small Language Model in Google Colab

## 1. Setup and Installation
First, we need to install the required libraries:

In [1]:
# Install the necessary libraries
!pip install transformers datasets torch accelerate

# transformers: Hugging Face's library for state-of-the-art NLP models
# datasets: Library for easily accessing and processing datasets
# torch: PyTorch deep learning framework
# accelerate: Library for distributed training

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import required libraries and set up GPU
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# This is important because training LLMs requires significant computational resources
# Even a small model will train much faster on GPU than CPU

Using device: cpu


## 2. Model and Tokenizer Initialization
We'll use GPT-2 small, which has 124M parameters:

In [3]:
# Load the model and tokenizer
model_name = "gpt2"  # This is the smallest GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Explanation:
# - GPT2Tokenizer: Converts text to tokens that the model can understand
# - GPT2LMHeadModel: The actual language model with a language modeling head
# - .to(device): Moves the model to GPU if available
# - pad_token: Required for batching sequences of different lengths

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## 3. Data Preparation
Let's load and prepare a small dataset (Shakespeare text as an example):

In [4]:
# Download the Shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=128):
        # Read the text file
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # Tokenize the entire text
        self.encodings = tokenizer(
            text,
            truncation=True,
            max_length=block_size,
            return_overflowing_tokens=True,
            return_length=True
        )

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# Create the dataset
dataset = TextDataset('input.txt', tokenizer)


--2025-02-08 18:19:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2025-02-08 18:19:27 (15.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



## 4. Training Configuration
Set up the training parameters and initialize the trainer:

In [7]:
# Configure training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-shakespeare",          # Directory to save model checkpoints
    overwrite_output_dir=True,               # Overwrite existing checkpoint directory
    num_train_epochs=1,                      # Number of training epochs
    per_device_train_batch_size=4,           # Batch size per GPU/CPU
    save_steps=500,                          # Save checkpoint every X steps
    save_total_limit=2,                      # Maximum number of checkpoints to keep
    logging_steps=100,                       # Log metrics every X steps
    learning_rate=3e-5,                      # Learning rate
    warmup_steps=500,                        # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                       # Weight decay for regularization
)

# Create data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're not using masked language modeling (like BERT)
)

# Initialize trainer
trainer = Trainer(
    model=model,                         # The instantiated model to be trained
    args=training_args,                  # Training arguments
    data_collator=data_collator,        # Data collator for creating batches
    train_dataset=dataset,              # Training dataset
)


## 5. Training Process
Now we can start the training:

In [8]:
# Start training
print("Starting training...")
trainer.train()
print("Training completed!")

# The trainer will:
# 1. Create batches from the dataset
# 2. Move data to GPU if available
# 3. Perform forward and backward passes
# 4. Update model parameters
# 5. Log metrics and save checkpoints

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting training...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: denny-wang-au (denny-wang-au-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


TypeError: 'int' object is not subscriptable

## 6. Inference
Test the trained model with text generation:

In [ ]:
def generate_text(prompt, max_length=100):
    """
    Generate text from a prompt using the trained model.

    Args:
        prompt (str): The input text to start generation from
        max_length (int): Maximum length of generated text

    Returns:
        str: Generated text
    """
    # Encode the prompt to token IDs
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate text
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Prevent repetition of n-grams
        temperature=0.7,         # Control randomness (higher = more random)
        top_k=50,               # Sample from top K most likely tokens
        top_p=0.95,            # Nucleus sampling parameter
    )

    # Decode the generated tokens back to text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
prompt = "To be or not to be"
generated_text = generate_text(prompt)
print(f"Prompt: {prompt}")
print(f"Generated: {generated_text}")

## 7. Model Evaluation and Monitoring

Let's add some basic monitoring of the model's performance:

In [ ]:
# Calculate perplexity on a test sequence
def calculate_perplexity(text):
    encodings = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**encodings)
        loss = outputs.loss
    return torch.exp(loss).item()

# Test perplexity
test_text = "Friends, Romans, countrymen, lend me your ears;"
perplexity = calculate_perplexity(test_text)
print(f"Perplexity on test text: {perplexity}")

# Lower perplexity indicates better model performance

## Important Notes and Best Practices:

1. **Memory Management**:
   - Monitor GPU memory usage in Colab
   - Use smaller batch sizes if running out of memory
   - Consider gradient checkpointing for larger models

2. **Training Time**:
   - Even small models can take significant time to train
   - Start with small datasets for testing
   - Increase dataset size gradually

3. **Model Size**:
   - GPT-2 small is already 124M parameters
   - Larger models need more GPU memory and training time
   - Consider using quantization for larger models

4. **Hyperparameter Tuning**:
   - Learning rate is crucial for stable training
   - Adjust batch size based on available memory
   - Monitor loss to detect training issues